In [6]:
import pandas as pd, numpy as np, pyodbc
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', 999)
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.theta import ThetaForecaster
import plotly.express as px; from plotly import graph_objects as go
import datetime as dt; ts = dt.datetime.now(); ts_str = ts.strftime("%Y_%m_%d_%H_%M_%S"); print(ts_str)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

2020_09_14_09_41_33


# Import

In [ ]:
%%time
con = (
    r'DRIVER=ODBC Driver 17 for SQL Server;'
    r'SERVER=172.31.228.90;'
    r'DATABASE=MPlanProd;'
    r'Trusted_Connection=no;'
    r'Uid=DataEng;'
    r'Pwd=Data@2020;'
)
sql = 'SELECT * FROM [MPlanProd].[dbo].[View_Transaction_SFT_6049]'
cxn = pyodbc.connect(con)
df1 = pd.read_sql(sql, cxn)
df  = df1.copy()
print(df1.shape, ', '.join(df1.columns))

In [ ]:
df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','ตาก','บางกระทุ่ม', 'CC6'])]
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Zone'] = df['Sloc'].str[:2]
ck = ['Zone', 'Sloc', 'SiteName',
      'Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton']
# WorkCentreName, 'Time', 'Ticket_no'
df[ck].to_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', index=False)
print(df.shape)

# Read

In [3]:
ts_str = '2020_09_13_16_06_52'
df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)
sloc = dict(df.groupby('SiteName')['Sloc'].max())
sloc = dict([(value, key) for key, value in sloc.items()])
zone = df.groupby('Zone').agg({'Sloc': lambda x: list(np.unique(x))})['Sloc'].to_dict()

In [4]:
df = df.drop(df.loc[(df.Year == dt.datetime.now().year) & (df.Month == dt.datetime.now().month)].index)
#df= df.Year.isin(['2016', '2017', '2018', '2019', '2020']).reset_index(drop=True)
df = df.loc[df.Year>=2016].reset_index(drop=True)
print(df.shape)
months = [dt.date(dt.datetime.now().year, i, 1).strftime('%b') for i in range(1, 13)]
df['Monthi'] = df['Month']
df['Month']  = df['Month'].apply(lambda x: months[int(x)-1])
df.Year      = df.Year.astype('str')
df.Month     = df.Month.astype('str')

(188733, 9)


# Cumulative

In [ ]:
ym, ymi, ymc, ymp = {}, {}, {}, {}
for s in ['all']:#+list(zone.keys())+list(sloc.keys()):
    if s == 'all':
        dfs = df.copy()
    elif len(s)==2:
        dfs = df.loc[df.Zone==s].copy()
    else:
        dfs = df.loc[df.Sloc==s].copy()
    ym[s], ymi[s], ymc[s] = {}, {}, {}
    for t in ['Purchase_price_thb_ton','MillWeight']:
        tcount   = dfs.groupby(['Year','Month','Monthi'])[t].count().reset_index()
        tmean    = dfs.groupby(['Year','Month','Monthi'])[t].mean().reset_index()
        tmerge   = tcount.merge(tmean, left_index=True, right_index=True, suffixes=('_m',''))
        #tmerge.rename(columns={t+'_m':'mean', t:'count'}, inplace=True)
        tmerge   = tmerge.rename(columns={t+'_m':'mean', t:'count'})[['mean','count']]
        ym[s][t] = dfs.groupby(['Year','Month','Monthi'])[t].sum().reset_index()
        ym[s][t] = ym[s][t].merge(tmerge, left_index=True,right_index=True,suffixes=('_cm',''))
        #ymi[s][t]= ym[s][t][ym[s][t].Year.isin(['2016', '2017', '2018', '2019', '2020'])].reset_index(drop=True)
        #ymi[s][t]['Monthi'] = pd.to_datetime(ymi[s][t].Month, format='%b', errors='coerce').dt.month
        #ymi[s][t].sort_values(by='Monthi', inplace=True)
        ymi[s][t]= ym[s][t].sort_values(by='Monthi')
        ymc[s][t]= ymi[s][t].groupby('Year').cumsum().merge(ymi[s][t], left_index=True,right_index=True,suffixes=('_c',''))
        ymc[s][t].drop(columns=['Monthi_c', 'mean_c', 'count_c'], inplace=True)
    
    ymp[s] = ymc[s]['Purchase_price_thb_ton'].merge(ymc[s]['MillWeight'],
                                                    left_on=['Year', 'Month', 'Monthi'],
                                                    right_on=['Year', 'Month', 'Monthi'])
    ymp[s]['Price']  = ymp[s]['Purchase_price_thb_ton'] / ymp[s]['MillWeight']
    ymp[s]['Price_c']= ymp[s]['Purchase_price_thb_ton_c'] / ymp[s]['MillWeight_c']
    #ymp[s].Year = ymp[s].Year.astype('str')
    #ymp[s].Month = ymp[s].Month.astype('str')

In [ ]:
for t in ['Purchase_price_thb_ton', 'MillWeight']:
    if t == 'MillWeight': cols = ['MillWeight', 'MillWeight_c']
    else: cols = ['Purchase_price_thb_ton', 'Purchase_price_thb_ton_c']
    for col in cols:
        fig = go.Figure()
        for year in ymc[s][t].Year.unique():
            Y  = ymc[s][t].loc[ymc[s][t].Year==year].copy()
            fig.add_trace(go.Scatter(x=Y['Month'], y=Y[col], mode="markers+text+lines", textposition="middle right",
                                     text=['' for i in range(len(Y)-1)]+[str(year)]))\
            #.update_xaxes(categoryorder='array', categoryarray=months)
        fig.update_layout(showlegend=False, title=col)
        fig.show()

In [ ]:
for col in ['Price', 'Price_c']:
    fig = go.Figure()
    for year in ymp[s].Year.unique():
        Y  = ymp[s].loc[ymp[s].Year==year]
        fig.add_trace(go.Scatter(x=Y['Month'], y=Y[col],
                                 mode="markers+text+lines", textposition="middle right",
                                 text=['' for i in range(len(Y)-1)]+[str(year)]))
    fig.update_layout(showlegend=False, title=col)
    fig.show()

# Wood Demand / Supply Model

In [ ]:
for s in ['all']:#+list(zone.keys())+list(sloc.keys()):
    f, forecaster = {}, {}; print(s)
    print(s, ymp[s].Month.unique())
    for m in ymp[s].Month.unique():
        if len(ymp[s].loc[ymp[s].Month == m]) > 1:
            
            # 2nd-Step: PQ-Regression
            f[m] = RandomForestRegressor()
            f[m].fit(ymp[s].loc[ymp[s].Month == m].MillWeight.values.reshape(-1, 1),
                     ymp[s].loc[ymp[s].Month == m]['Price'].values)
            
            # 1st-Step: TS-Forecasting
            hist = ymp[s].loc[ymp[s].Month == m].sort_values(by='Year')
            hist = pd.Series(hist['MillWeight'].astype(float).values)
            print(hist)
            forecaster[m] = ThetaForecaster(sp=1)
            forecaster[m].fit(hist)
            yp = forecaster[m].predict([1]).values[0]
            if m in ymp[s].loc[ymp[s].Year == str(dt.datetime.now().year), 'Month'].values:
                ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                                  'Year': str(dt.datetime.now().year + 1) + 'F',
                                                  'MillWeight': yp}), ignore_index=True)
            else:
                ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                                  'Year': str(dt.datetime.now().year) + 'F',
                                                  'MillWeight': yp}), ignore_index=True)
                yp = forecaster[m].update_predict_single(pd.Series(yp), fh=[1]).values[0]
                ymp[s] = ymp[s].append(pd.Series({'Month': m,
                                                  'Year': str(dt.datetime.now().year + 1) + 'F',
                                                  'MillWeight': yp}), ignore_index=True)

                x_pred = ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F') &
                                    (ymp[s].Month == m), 'MillWeight'].values[0]
                y_pred = f[m].predict(x_pred.reshape(1, -1)); print(1, x_pred, y_pred, '\n')
                ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F')&
                           (ymp[s].Month == m), 'Price'] = y_pred
                
                for cm in ['mean_x','mean_y','count_x','count_y']:
                    ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year) + 'F')&
                               (ymp[s].Month == m), cm] = ymp[s].loc[ymp[s].Month == m, cm].mean()

            x_pred = ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F') &
                                (ymp[s].Month == m), 'MillWeight'].values[0]
            y_pred = f[m].predict(x_pred.reshape(1, -1)); print(2, x_pred, y_pred, '\n')
            ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F')&
                       (ymp[s].Month == m), 'Price'] = y_pred
            
            for cm in ['mean_x','mean_y','count_x','count_y']:
                ymp[s].loc[(ymp[s].Year == str(dt.datetime.now().year + 1) + 'F')&
                           (ymp[s].Month == m), cm] = ymp[s].loc[ymp[s].Month == m, cm].mean()

In [ ]:
ympd = {}
for s in ['all']:#+list(zone.keys())+list(sloc.keys()):
    a = ymp[s].copy()
    a['Y'] = 'Year'
    a['YP'] = a['Year'].str.replace('F','').astype(int) - 2000
    a['YC'] = a['YP']
    a['Type'] = '2016_2019_Actual'
    a.loc[a['Year']=='2020', 'Type'] = '2020_Actual'
    a.loc[a['Year']=='2020F', 'Type'] = '2020_Forecast'
    a.loc[a['Year']=='2021F', 'Type'] = '2021_Forecast'
    b = ymp[s].copy()
    b['Y'] = 'Price'
    b['YP'] = b['Price']
    b['YC'] = b['Year'].str.replace('F','').astype(int)
    b['Type'] = '2016_2019_Actual'
    b.loc[b['Year']=='2020', 'Type'] = '2020_Actual'
    b.loc[b['Year']=='2020F', 'Type'] = '2020_Forecast'
    b.loc[b['Year']=='2021F', 'Type'] = '2021_Forecast'
    ympd[s] = b.append(a, ignore_index=True)

In [ ]:
#ymp['all'].loc[ymp['all'].Month == 'Sep']
#ymp['all']
ympd['all']

In [ ]:
print('Top: Step_1_Forecast Quantity (Size=Average Transaction Price) | Bottom: Step_2_Forecast Price (Size=Number of Transactions) | Color: Year (Dasker=Newer)', end='')
fig1 = px.scatter(ympd['all'].loc[ympd['all'].Y=='Year'], x='MillWeight', y='YP', color='YC', size='mean_x', symbol='Type',
                 color_continuous_scale='Blugrn', facet_col_spacing=0.005, facet_col='Month', height=240)
fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig1.for_each_yaxis(lambda a: a.update(dtick=1))
fig1.for_each_yaxis(lambda a: a.update(title='', tickangle=90, dtick=1), row=1, col=1)
fig1.for_each_xaxis(lambda a: a.update(title='', dtick=10000, showticklabels=False)) # showgrid=True, visible=False
fig1.layout.update(coloraxis_showscale=False, showlegend=False, margin=dict(l=0,r=0,t=15,b=0))
fig1.update_yaxes(autorange="reversed") # matches=None
fig1.show()

fig2 = px.scatter(ympd['all'].loc[ympd['all'].Y=='Price'], x='MillWeight', y='YP', color='YC', size='count_x',symbol='Type',# text='MillWeight',
                 color_continuous_scale='Blugrn', facet_col_spacing=0.005, facet_col='Month', height=500, labels={'Type':''}) #, facet_row='Y'
fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig2.for_each_yaxis(lambda a: a.update(title='', tickangle=90), row=1, col=1)
fig2.for_each_xaxis(lambda a: a.update(title='', dtick=10000))
#fig2.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig2.layout.update(coloraxis_showscale=False, legend_orientation='h', margin=dict(l=0, r=0, t=0, b=0))
fig2.show() #text=['' for i in range(len(Y)-1)]+[str(year)]

In [ ]:
ympd['all']

In [ ]:
ympd['all'].loc[(ympd['all'].Month == m) & (ympd['all'].YC == 2020), 'Price'].iloc[0]

In [ ]:
f"{:,}"

# Export

In [33]:
import json

In [ ]:
wp = {}
for s in sloc.keys():
    df_ = df.loc[df.Sloc==s].groupby(['Year', 'Monthi', 'Day']).sum().reset_index()
    df_['Price_Daily'] = df_['Purchase_price_thb_ton']/df_['MillWeight']
    df_.drop('Price_ton', axis=1 ,inplace=True)
    wp[s] = df_.to_json(orient = 'records')

In [ ]:
with open('Sloc.json', 'w') as o: json.dump(sloc, o)
with open('Zone.json', 'w') as o: json.dump(zone, o)
with open('wp.json', 'w') as o: json.dump(wp, o)

In [ ]:
wpm = {}
with open('wp.json', 'r') as o: wp = json.load(o)
for s in sloc.keys():
    wp[s]          = pd.read_json(wp[s], orient = 'records').rename(columns={'Monthi':'Month'})
    wp[s]['dt']    = pd.to_datetime(wp[s][['Year', 'Month', 'Day']])
    #wp[s].drop(['Year', 'Month', 'Day'], axis=1, inplace=True)
    wpm[s]         = wp[s].groupby(['Year','Month']).sum().reset_index().drop('Price_Daily', axis=1)
    wpm[s]['Price_Monthly'] = wpm[s]['Purchase_price_thb_ton']/wpm[s]['MillWeight']
    wpm[s]['Day']  = 1
    wpm[s]['dt']   = pd.to_datetime(wpm[s][['Year', 'Month', 'Day']])
    wpm[s]['Sloc'] = s

# Test

In [34]:
with open('ympd.json', 'r') as o:
    ympd = json.load(o)
for s in ['all'] + list(zone.keys()) + list(sloc.keys()):
    ympd[s] = pd.read_json(ympd[s], orient='records')

In [37]:
ympd.keys()

dict_keys(['all', 'CH', 'N1', 'N2', 'N3', 'W1', 'W1CP', 'N3CD', 'N1DK', 'N3DT', 'W1DT', 'N1TN', 'N1TC', 'N3TK', 'N1BK', 'N2MK', 'N2ST', 'N1PR', 'N1PB', 'N1YS', 'W1NS', 'N2NS', 'N2WP', 'N1WT', 'N3WR', 'N1DG', 'N2SB', 'W1SR', 'N3NB', 'N1NK', 'N2KP', 'CH11', 'CH12', 'CH13'])

In [38]:
ympd['CH']

,Purchase_price_thb_ton_c,Year,Month,Monthi,Purchase_price_thb_ton,mean_x,count_x,MillWeight_c,MillWeight,mean_y,count_y,Price,Price_c,Y,YP,YC,Type
0,"16,077,319.4000",2019,Jan,1.0000,"16,077,319.4000","2,905.0000","5,534.3612","29,556.5100","29,556.5100","2,905.0000",10.1744,543.9519,543.9519,Price,543.9519,2019,2016_2019_Actual
1,"20,768,060.6000",2020,Jan,1.0000,"20,768,060.6000","4,976.0000","4,173.6456","23,436.7100","23,436.7100","4,976.0000",4.7099,886.1338,886.1338,Price,886.1338,2020,2020_Actual
2,"16,586,545.1800",2018,Jan,1.0000,"16,586,545.1800","3,020.0000","5,492.2335","21,869.7400","21,869.7400","3,020.0000",7.2416,758.4244,758.4244,Price,758.4244,2018,2016_2019_Actual
3,"16,882,399.7000",2016,Jan,1.0000,"16,882,399.7000","2,561.0000","6,592.1123","15,406.8900","15,406.8900","2,561.0000",6.0160,"1,095.7695","1,095.7695",Price,"1,095.7695",2016,2016_2019_Actual
4,"17,790,016.8500",2017,Jan,1.0000,"17,790,016.8500","2,079.0000","8,557.0067","18,762.9100","18,762.9100","2,079.0000",9.0250,948.1481,948.1481,Price,948.1481,2017,2016_2019_Actual
5,"39,927,991.7700",2017,Feb,2.0000,"22,137,974.9200","3,164.0000","6,996.8315","40,272.1300","21,509.2200","3,164.0000",6.7981,"1,029.2319",991.4547,Price,"1,029.2319",2017,2016_2019_Actual
6,"30,169,406.4000",2019,Feb,2.0000,"14,092,087.0000","2,650.0000","5,317.7687","57,376.8500","27,820.3400","2,650.0000",10.4982,506.5390,525.8115,Price,506.5390,2019,2016_2019_Actual
7,"30,963,499.0500",2018,Feb,2.0000,"14,376,953.8700","3,061.0000","4,696.8160","43,255.3300","21,385.5900","3,061.0000",6.9865,672.2730,715.8308,Price,672.2730,2018,2016_2019_Actual
8,"37,115,707.1000",2016,Feb,2.0000,"20,233,307.4000","2,996.0000","6,753.4404","33,923.6400","18,516.7500","2,996.0000",6.1805,"1,092.7030","1,094.0957",Price,"1,092.7030",2016,2016_2019_Actual
9,"43,456,951.0000",2020,Feb,2.0000,"22,688,890.4000","4,375.0000","5,186.0321","46,617.8900","23,181.1800","4,375.0000",5.2986,978.7634,932.1947,Price,978.7634,2020,2020_Actual


In [ ]:
wp[s]

In [ ]:
wpms = {}
for s in sloc.keys():
    sen        = wpm[s][['MillWeight','Price_Monthly']].pct_change().apply(lambda x: round(x, 5))*100
    sen['sen'] = sen['MillWeight'] / sen['Price_Monthly']
    sen        = sen.replace(np.inf, 1000).replace(-np.inf, -1000).clip(-100, 100)
    wpms[s]    = sen.merge(wpm[s]['dt'], left_index=True, right_index=True)
    wpms[s]['Sloc'] = s

In [ ]:
wpmj, wpmsj = {}, {};
wpmsjz = pd.DataFrame()
for z in zone.keys():
    a, b = pd.DataFrame(), pd.DataFrame()
    for s in zone[z]:
        a = a.append(wpm[s])
        b = b.append(wpms[s])
    wpmj[z] = a
    wpmsj[z] = b
    wpmsjz = wpmsjz.append(wpmsj[z])

In [ ]:
tl = np.sort(wpmsjz.dt.unique())[::-1]
ss = wpmsjz.Sloc.unique()
dft= pd.DataFrame()
for t in tl:
    dfs = pd.DataFrame(ss, columns=['Sloc'])
    dfs['dt'] = t
    dft = dft.append(dfs)
wpmsjzm = wpmsjz.merge(dft, left_on=['dt','Sloc'], right_on=['dt','Sloc'], how='outer')


In [ ]:
len(px.scatter(wpmj[z], x='dt', y='Price_Monthly', size='MillWeight', color='Sloc', size_max=60).data)

In [ ]:
px.line(wpms[s], x='dt', y='sen')